In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sarcoma"
cohort = "GSE133228"

# Input paths
in_trait_dir = "../../input/GEO/Sarcoma"
in_cohort_dir = "../../input/GEO/Sarcoma/GSE133228"

# Output paths
out_data_file = "../../output/preprocess/Sarcoma/GSE133228.csv"
out_gene_data_file = "../../output/preprocess/Sarcoma/gene_data/GSE133228.csv"
out_clinical_data_file = "../../output/preprocess/Sarcoma/clinical_data/GSE133228.csv"
json_path = "../../output/preprocess/Sarcoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE133228-GPL16311_series_matrix.txt.gz', 'GSE133228_family.soft.gz']
SOFT file: ../../input/GEO/Sarcoma/GSE133228/GSE133228_family.soft.gz
Matrix file: ../../input/GEO/Sarcoma/GSE133228/GSE133228-GPL16311_series_matrix.txt.gz
Background Information:
!Series_title	"STAG2 promotes CTCF-anchored loop extrusion and cis-promoter and -enhancer interactions"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['gender: Male', 'gender: Female'], 1: ['age: 3', 'age: 11', 'age: 4', 'age: 25', 'age: 13', 'age: 15', 'age: 19', 'age: 8', 'age: 20', 'age: 24', 'age: 16', 'age: 14', 'age: 5', 'age: 37', 'age: 26', 'age: 10', 'age: 35', 'age: 23', 'age: 17', 'age: 12', 'age: 9', 'age: 0', 'age: 36', 'age: 27', 'age: 1', 'age: 18', 'age: 29', 'age: 6', 'age: 28', 'age: 31'], 2: ['tumor type: primary tumor']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import re
import numpy as np

# Load clinical data
clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "GSE133228-GPL16311_series_matrix.txt.gz"), 
                            sep='\t', comment='!', skiprows=0)

# 1. Gene Expression Data Availability
# Based on the matrix filename "GSE133228-GPL16311_series_matrix.txt.gz", this likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Sarcoma)
# From sample characteristics dict, key 2 has 'tumor type: primary tumor', but it's a constant value
# As per instructions, constant features are useless in associative studies
trait_row = None

# For age
# Age is available under key 1 in the sample characteristics dictionary
age_row = 1

# For gender
# Gender is available under key 0 in the sample characteristics dictionary
gender_row = 0

# 2.2 Data Type Conversion Functions

# For trait (keeping this function in case it's needed later)
def convert_trait(value):
    if value is None:
        return None
    
    # Handle if value is already numeric
    if isinstance(value, (int, float)):
        return 1 if value == 1 else 0
    
    # For string values, extract after colon if present
    if ':' in str(value):
        value = str(value).split(':', 1)[1].strip()
    
    if 'primary tumor' in str(value).lower():
        return 1
    else:
        return 0

# For age
def convert_age(value):
    if value is None:
        return None
    
    # Handle if value is already numeric
    if isinstance(value, (int, float)):
        return float(value)
    
    # Extract value after colon if present
    if ':' in str(value):
        value = str(value).split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

# For gender
def convert_gender(value):
    if value is None:
        return None
    
    # Handle if value is already numeric
    if isinstance(value, (int, float)):
        return 1 if value == 1 else 0 if value == 0 else None
    
    # Extract value after colon if present
    if ':' in str(value):
        value = str(value).split(':', 1)[1].strip().lower()
    else:
        value = str(value).lower()
    
    if 'female' in value:
        return 0
    elif 'male' in value:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we skip this step
if trait_row is not None:
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
else:
    print("Trait data is not available. Skipping clinical feature extraction.")


Trait data is not available. Skipping clinical feature extraction.


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
Found potential subseries references:
!Series_relation = SuperSeries of: GSE132966
!Series_relation = SuperSeries of: GSE133154
!Series_relation = SuperSeries of: GSE133227
!Series_relation = SuperSeries of: GSE142162
!Series_relation = SuperSeries of: GSE156649
!Series_relation = SuperSeries of: GSE156650
!Series_relation = SuperSeries of: GSE156653
!Series_relation = SuperSeries of: GSE171948

Gene data extraction result:
Number of rows: 19070
First 20 gene/probe identifiers:
Index(['100009676_at', '10000_at', '10001_at', '10002_at', '10003_at',
       '100048912_at', '100049716_at', '10004_at', '10005_at', '10006_at',
       '10007_at', '10008_at', '100093630_at', '10009_at', '1000_at',
       '100101467_at', '100101938_at', '10010_at', '100113407_at', '10011_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyze the gene identifiers
# The format "XXX_at" where XXX is a numerical ID suggests these are probe identifiers
# from a microarray platform (likely Affymetrix), not standard human gene symbols.
# These need to be mapped to official gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1_at', '10_at', '100_at', '1000_at', '10000_at'], 'SPOT_ID': ['1', '10', '100', '1000', '10000'], 'Description': ['alpha-1-B glycoprotein', 'N-acetyltransferase 2 (arylamine N-acetyltransferase)', 'adenosine deaminase', 'cadherin 2, type 1, N-cadherin (neuronal)', 'v-akt murine thymoma viral oncogene homolog 3 (protein kinase B, gamma)']}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation dataframe contain the identifiers and symbols
# From the preview, we can see:
# - 'ID' column contains identifiers like '1_at', matching the gene expression data format
# - 'Description' column contains gene names/descriptions

# 2. Extract the gene mapping dataframe with probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Description')

# Print info about the mapping
print("Gene mapping info:")
print("Total number of probe-gene mappings:", len(gene_mapping))
print("Sample mappings (first 5 rows):")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)

# Print some statistics about the gene data after mapping
print("\nAfter mapping:")
print("Number of unique genes:", len(gene_data))
print("First 5 gene symbols:")
print(gene_data.index[:5])


Gene mapping info:
Total number of probe-gene mappings: 19037
Sample mappings (first 5 rows):
         ID                                               Gene
0      1_at                             alpha-1-B glycoprotein
1     10_at  N-acetyltransferase 2 (arylamine N-acetyltrans...
2    100_at                                adenosine deaminase
3   1000_at          cadherin 2, type 1, N-cadherin (neuronal)
4  10000_at  v-akt murine thymoma viral oncogene homolog 3 ...

After mapping:
Number of unique genes: 2034
First 5 gene symbols:
Index(['A-', 'A-2', 'A-52', 'A-I', 'A-II'], dtype='object', name='Gene')


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols - let's take a more memory-efficient approach
# Instead of doing all at once, process in smaller chunks
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get fresh gene expression data
gene_data = get_genetic_data(matrix_file)
print(f"Original gene expression data shape: {gene_data.shape}")

# Get the gene annotation again
gene_annotation = get_gene_annotation(soft_file)
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Description')
print(f"Created mapping with {len(gene_mapping)} entries")

# Process and map in chunks to reduce memory usage
batch_size = 1000
num_batches = (len(gene_data) + batch_size - 1) // batch_size
result_dfs = []

for i in range(num_batches):
    print(f"Processing batch {i+1}/{num_batches}")
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(gene_data))
    
    # Get a subset of the expression data
    batch_expr = gene_data.iloc[start_idx:end_idx]
    
    # Process this batch
    batch_mapping = gene_mapping[gene_mapping['ID'].isin(batch_expr.index)]
    if len(batch_mapping) > 0:
        mapped_batch = apply_gene_mapping(batch_expr, batch_mapping)
        result_dfs.append(mapped_batch)
    
    # Clear memory
    del batch_expr
    del batch_mapping

# Combine results
if result_dfs:
    mapped_gene_data = pd.concat(result_dfs)
    print(f"After mapping: {mapped_gene_data.shape}")
    
    # Normalize gene symbols using NCBI database
    try:
        gene_data_normalized = normalize_gene_symbols_in_index(mapped_gene_data)
        print(f"After normalization: {gene_data_normalized.shape}")
    except Exception as e:
        print(f"Error during normalization: {e}")
        # Fallback to unmapped data
        gene_data_normalized = mapped_gene_data
else:
    print("Mapping failed for all batches, using original data")
    gene_data_normalized = gene_data

# Save the gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# 2. Create clinical data with the trait information
sample_ids = gene_data.columns.tolist()
print(f"Sample IDs from gene data: {len(sample_ids)} samples")

# Create a clinical dataframe with the trait (Sarcoma)
clinical_df = pd.DataFrame(index=[trait], columns=sample_ids)
clinical_df.loc[trait] = 1  # All samples are sarcoma tumors

print(f"Clinical data shape: {clinical_df.shape}")

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 3. Link clinical and genetic data - using smaller version for efficiency
# Select a subset of genes to reduce memory issues
print("Selecting top 5000 genes with highest variance...")
if len(gene_data_normalized) > 5000:
    gene_variance = gene_data_normalized.var(axis=1)
    top_genes = gene_variance.nlargest(5000).index
    gene_data_subset = gene_data_normalized.loc[top_genes]
else:
    gene_data_subset = gene_data_normalized

print(f"Subset gene data shape: {gene_data_subset.shape}")
linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data_subset)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
print("Handling missing values...")
linked_data_cleaned = handle_missing_values(linked_data, trait)
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait and demographic features are biased
print("Checking for bias in features...")
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)

# 6. Validate the dataset and save cohort information
note = "Dataset contains expression data for pediatric tumors including rhabdomyosarcoma (sarcoma). All samples are tumor samples, so trait bias is expected and the dataset is not suitable for case-control analysis."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_trait_biased,
    df=unbiased_linked_data,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to trait bias. Final linked data not saved.")

Original gene expression data shape: (19070, 79)


Created mapping with 19037 entries
Processing batch 1/20
Processing batch 2/20
Processing batch 3/20
Processing batch 4/20
Processing batch 5/20
Processing batch 6/20
Processing batch 7/20
Processing batch 8/20
Processing batch 9/20
Processing batch 10/20
Processing batch 11/20
Processing batch 12/20
Processing batch 13/20
Processing batch 14/20


Processing batch 15/20
Processing batch 16/20
Processing batch 17/20
Processing batch 18/20
Processing batch 19/20
Processing batch 20/20
After mapping: (4280, 79)
After normalization: (1171, 79)
Gene expression data saved to ../../output/preprocess/Sarcoma/gene_data/GSE133228.csv
Sample IDs from gene data: 79 samples
Clinical data shape: (1, 79)


Clinical data saved to ../../output/preprocess/Sarcoma/clinical_data/GSE133228.csv
Selecting top 5000 genes with highest variance...
Subset gene data shape: (1171, 79)
Shape of linked data: (79, 1172)
Handling missing values...
Shape of linked data after handling missing values: (79, 1172)
Checking for bias in features...
Quartiles for 'Sarcoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Sarcoma' in this dataset is severely biased.



/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())


Dataset validation failed due to trait bias. Final linked data not saved.
